# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [2]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import os

We set the random seed so that the results don't vary drastically.

In [3]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

Using TensorFlow backend.


In [4]:
import cv2
import matplotlib.pyplot as plt
% matplotlib inline

In [5]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv3D, MaxPooling3D, Conv2D, MaxPooling2D
from keras.layers.recurrent import LSTM
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from keras.layers import Dropout

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [6]:
train_doc = np.random.permutation(open('Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('Project_data/val.csv').readlines())
project_folder='Project_data'


## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [11]:
class ModelBuilder():
    
    def initialize_path(self,project_folder):
        self.train_doc = np.random.permutation(open(project_folder + '/' + 'train.csv').readlines())
        self.val_doc = np.random.permutation(open(project_folder + '/' + 'val.csv').readlines())
        self.train_path = project_folder + '/' + 'train'
        self.val_path =  project_folder + '/' + 'val'
        self.num_train_sequences = len(self.train_doc)
        self.num_val_sequences = len(self.val_doc)
        
    def initialize_image_properties(self,image_height=100,image_width=100):
        self.image_height=image_height
        self.image_width=image_width
        self.channels=3
        self.num_classes=5
        self.total_frames=30
          
    def initialize_hyperparams(self,frames_to_sample=30,batch_size=20,num_epochs=20):
        self.frames_to_sample=frames_to_sample
        self.batch_size=batch_size
        self.num_epochs=num_epochs
        
        
    def generator(self,source_path, folder_list, augment=False):
        img_idx = np.round(np.linspace(0,self.total_frames-1,self.frames_to_sample)).astype(int)
        batch_size=self.batch_size
        while True:
            t = np.random.permutation(folder_list)
            num_batches = len(t)//batch_size
        
            for batch in range(num_batches): 
                batch_data, batch_labels= self.one_batch_data(source_path,t,batch,batch_size,img_idx,augment)
                yield batch_data, batch_labels 

            remaining_seq=len(t)%batch_size
        
            if (remaining_seq != 0):
                batch_data, batch_labels= self.one_batch_data(source_path,t,num_batches,batch_size,img_idx,augment,remaining_seq)
                yield batch_data, batch_labels 
    
    
    def one_batch_data(self,source_path,t,batch,batch_size,img_idx,augment,remaining_seq=0):
    
        seq_len = remaining_seq if remaining_seq else batch_size
    
        batch_data = np.zeros((seq_len,len(img_idx),self.image_height,self.image_width,self.channels)) 
        batch_labels = np.zeros((seq_len,self.num_classes)) 
    
        if (augment): batch_data_aug = np.zeros((seq_len,len(img_idx),self.image_height,self.image_width,self.channels))

        
        for folder in range(seq_len): 
            imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) 
            for idx,item in enumerate(img_idx): 
                image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                image_resized=imresize(image,(self.image_height,self.image_width,3))
            

                batch_data[folder,idx,:,:,0] = (image_resized[:,:,0])/255
                batch_data[folder,idx,:,:,1] = (image_resized[:,:,1])/255
                batch_data[folder,idx,:,:,2] = (image_resized[:,:,2])/255
            
                if (augment):
                    shifted = cv2.warpAffine(image, 
                                             np.float32([[1, 0, np.random.randint(-30,30)],[0, 1, np.random.randint(-30,30)]]), 
                                            (image.shape[1], image.shape[0]))
                    
                    gray = cv2.cvtColor(shifted,cv2.COLOR_BGR2GRAY)

                    x0, y0 = np.argwhere(gray > 0).min(axis=0)
                    x1, y1 = np.argwhere(gray > 0).max(axis=0) 
                    
                    cropped=shifted[x0:x1,y0:y1,:]
                    
                    image_resized=imresize(cropped,(self.image_height,self.image_width,3))
                    
                         
                    batch_data_aug[folder,idx,:,:,0] = (image_resized[:,:,0])/255
                    batch_data_aug[folder,idx,:,:,1] = (image_resized[:,:,1])/255
                    batch_data_aug[folder,idx,:,:,2] = (image_resized[:,:,2])/255
                
            
            batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            
    
        if (augment):
            batch_data=np.concatenate([batch_data,batch_data_aug])
            batch_labels=np.concatenate([batch_labels,batch_labels])

        
        return(batch_data,batch_labels)
    
    
    def train_model(self, model, augment_data=False):
        train_generator = self.generator(self.train_path, self.train_doc,augment=augment_data)
        val_generator = self.generator(self.val_path, self.val_doc)

        model_name = 'model_init' + '_' + str(datetime.datetime.now()).replace(' ','').replace(':','_') + '/'
    
        if not os.path.exists(model_name):
            os.mkdir(model_name)
        
        filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

        checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)
        LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=4)
        callbacks_list = [checkpoint, LR]

        if (self.num_train_sequences%self.batch_size) == 0:
            steps_per_epoch = int(self.num_train_sequences/self.batch_size)
        else:
            steps_per_epoch = (self.num_train_sequences//self.batch_size) + 1

        if (self.num_val_sequences%self.batch_size) == 0:
            validation_steps = int(self.num_val_sequences/self.batch_size)
        else:
            validation_steps = (self.num_val_sequences//self.batch_size) + 1
    
        history=model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=self.num_epochs, verbose=1, 
                            callbacks=callbacks_list, validation_data=val_generator, 
                            validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)
        return history


### Define a function for Plotting

In [12]:
def plot(history):
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15,4))
    axes[0].plot(history.history['loss'])   
    axes[0].plot(history.history['val_loss'])
    axes[0].legend(['loss','val_loss'])

    axes[1].plot(history.history['categorical_accuracy'])   
    axes[1].plot(history.history['val_categorical_accuracy'])
    axes[1].legend(['categorical_accuracy','val_categorical_accuracy'])

## Conv 3D : Model 1
Image : 160 * 160, Frames : 30 , Epochs : 1, Batch Size : 40

In [13]:
class ModelConv3D1(ModelBuilder):
    
    def define_model(self):

        model = Sequential()
        model.add(Conv3D(16, (3, 3, 3), padding='same',
                 input_shape=(self.frames_to_sample,self.image_height,self.image_width,self.channels)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))

        model.add(Conv3D(32, (2, 2, 2), padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))

        model.add(Conv3D(64, (2, 2, 2), padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))

        model.add(Conv3D(128, (2, 2, 2), padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))

        model.add(Flatten())
        model.add(Dense(128,activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))

        model.add(Dense(64,activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.25))


        model.add(Dense(self.num_classes,activation='softmax'))

        optimiser = optimizers.Adam()
        model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
        return model

In [13]:
conv_3d1=ModelConv3D1()
conv_3d1.initialize_path(project_folder)
conv_3d1.initialize_image_properties(image_height=160,image_width=160)
conv_3d1.initialize_hyperparams(frames_to_sample=30,batch_size=40,num_epochs=1)
conv_3d1_model=conv_3d1.define_model()
print("Total Params:", conv_3d1_model.count_params())
conv_3d1_model.summary()
model1 = conv_3d1.train_model(conv_3d1_model)

Total Params: 1736389
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 30, 160, 160, 16)  1312      
_________________________________________________________________
activation_1 (Activation)    (None, 30, 160, 160, 16)  0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 160, 160, 16)  64        
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 15, 80, 80, 16)    0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 15, 80, 80, 32)    4128      
_________________________________________________________________
activation_2 (Activation)    (None, 15, 80, 80, 32)    0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 15, 80, 80, 32)   

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:55: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:56: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


ResourceExhaustedError: OOM when allocating tensor with shape[40,16,30,160,160] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: training/Adam/gradients/max_pooling3d_1/MaxPool3D_grad/MaxPool3DGrad = MaxPool3DGrad[T=DT_FLOAT, TInput=DT_FLOAT, _class=["loc:@training/Adam/gradients/batch_normalization_1/cond/Merge_grad/cond_grad"], data_format="NDHWC", ksize=[1, 2, 2, 2, 1], padding="VALID", strides=[1, 2, 2, 2, 1], _device="/job:localhost/replica:0/task:0/device:GPU:0"](batch_normalization_1/cond/Merge, max_pooling3d_1/MaxPool3D, training/Adam/gradients/conv3d_2/convolution_grad/Conv3DBackpropInputV2)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


### The Model throws Out of Memory, when the batch size is 40 and epoch is 1.
In Next Model, batch size can be reduced.

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

### Conv 3D Model 2 (Architecture same as Model1 with change in HyperParameters)
Image : 160 * 160, Frames : 30 , Epochs : 1, Batch Size : 30

In [14]:
conv_3d2=ModelConv3D1()
conv_3d2.initialize_path(project_folder)
conv_3d2.initialize_image_properties(image_height=160,image_width=160)
conv_3d2.initialize_hyperparams(frames_to_sample=30,batch_size=30,num_epochs=1)
conv_3d2_model=conv_3d2.define_model()
conv_3d2_model.summary()
print("Total Params:", conv_3d2_model.count_params())
history_model2 = conv_3d2.train_model(conv_3d2_model)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_5 (Conv3D)            (None, 30, 160, 160, 16)  1312      
_________________________________________________________________
activation_5 (Activation)    (None, 30, 160, 160, 16)  0         
_________________________________________________________________
batch_normalization_7 (Batch (None, 30, 160, 160, 16)  64        
_________________________________________________________________
max_pooling3d_5 (MaxPooling3 (None, 15, 80, 80, 16)    0         
_________________________________________________________________
conv3d_6 (Conv3D)            (None, 15, 80, 80, 32)    4128      
_________________________________________________________________
activation_6 (Activation)    (None, 15, 80, 80, 32)    0         
_________________________________________________________________
batch_normalization_8 (Batch (None, 15, 80, 80, 32)    128       
__________

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:55: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:56: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


23/23 [==============================] - 168s 7s/step - loss: 1.5901 - categorical_accuracy: 0.3894 - val_loss: 1.2286 - val_categorical_accuracy: 0.5400

Epoch 00001: saving model to model_init_2021-02-0807_13_19.506464/model-00001-1.59621-0.37858-1.22860-0.54000.h5


### Conclusion : 
Train and Validation Accuracy is very low i.e. 38% and 54% respectively.

### Model 3 (Architecture same as Model1 with change in HyperParameters)
Image : 160 * 160, Frames : 30 , Epochs : 10, Batch Size : 30

In [19]:
conv_3d3=ModelConv3D1()
conv_3d3.initialize_path(project_folder)
conv_3d3.initialize_image_properties(image_height=160,image_width=160)
conv_3d3.initialize_hyperparams(frames_to_sample=30,batch_size=30,num_epochs=10)
conv_3d3_model=conv_3d3.define_model()
conv_3d3_model.summary()
print("Total Params:", conv_3d3_model.count_params())
history_model3 = conv_3d3.train_model(conv_3d3_model)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_9 (Conv3D)            (None, 30, 160, 160, 16)  1312      
_________________________________________________________________
activation_9 (Activation)    (None, 30, 160, 160, 16)  0         
_________________________________________________________________
batch_normalization_13 (Batc (None, 30, 160, 160, 16)  64        
_________________________________________________________________
max_pooling3d_9 (MaxPooling3 (None, 15, 80, 80, 16)    0         
_________________________________________________________________
conv3d_10 (Conv3D)           (None, 15, 80, 80, 32)    4128      
_________________________________________________________________
activation_10 (Activation)   (None, 15, 80, 80, 32)    0         
_________________________________________________________________
batch_normalization_14 (Batc (None, 15, 80, 80, 32)    128       
__________

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:55: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:56: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


23/23 [==============================] - 100s 4s/step - loss: 1.4850 - categorical_accuracy: 0.4402 - val_loss: 1.2746 - val_categorical_accuracy: 0.5700

Epoch 00001: saving model to model_init_2021-02-0807_22_27.046052/model-00001-1.51240-0.43137-1.27456-0.57000.h5
Epoch 2/10
23/23 [==============================] - 86s 4s/step - loss: 1.0480 - categorical_accuracy: 0.5874 - val_loss: 1.1201 - val_categorical_accuracy: 0.5200

Epoch 00002: saving model to model_init_2021-02-0807_22_27.046052/model-00002-1.01897-0.59729-1.12014-0.52000.h5
Epoch 3/10
23/23 [==============================] - 88s 4s/step - loss: 0.8320 - categorical_accuracy: 0.6609 - val_loss: 0.7990 - val_categorical_accuracy: 0.6600

Epoch 00003: saving model to model_init_2021-02-0807_22_27.046052/model-00003-0.84227-0.66063-0.79899-0.66000.h5
Epoch 4/10
23/23 [==============================] - 89s 4s/step - loss: 0.8334 - categorical_accuracy: 0.6556 - val_loss: 1.2093 - val_categorical_accuracy: 0.5400

Epoch 00004

### Conclusion : The accuracy increased to 83/80 % with number of epochs but execution time also increases.

### Model 4 (Architecture same as Model1 with change in HyperParameters)
Image : 160 * 160, Frames : 30 , Epochs : 30, Batch Size : 30

In [20]:
conv_3d4=ModelConv3D1()
conv_3d4.initialize_path(project_folder)
conv_3d4.initialize_image_properties(image_height=160,image_width=160)
conv_3d4.initialize_hyperparams(frames_to_sample=30,batch_size=30,num_epochs=30)
conv_3d4_model=conv_3d4.define_model()
conv_3d4_model.summary()
print("Total Params:", conv_3d4_model.count_params())
history_model4 = conv_3d4.train_model(conv_3d4_model)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_13 (Conv3D)           (None, 30, 160, 160, 16)  1312      
_________________________________________________________________
activation_13 (Activation)   (None, 30, 160, 160, 16)  0         
_________________________________________________________________
batch_normalization_19 (Batc (None, 30, 160, 160, 16)  64        
_________________________________________________________________
max_pooling3d_13 (MaxPooling (None, 15, 80, 80, 16)    0         
_________________________________________________________________
conv3d_14 (Conv3D)           (None, 15, 80, 80, 32)    4128      
_________________________________________________________________
activation_14 (Activation)   (None, 15, 80, 80, 32)    0         
_________________________________________________________________
batch_normalization_20 (Batc (None, 15, 80, 80, 32)    128       
__________

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:55: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:56: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


23/23 [==============================] - 100s 4s/step - loss: 1.7455 - categorical_accuracy: 0.3856 - val_loss: 2.0531 - val_categorical_accuracy: 0.3700

Epoch 00001: saving model to model_init_2021-02-0807_43_26.250032/model-00001-1.74571-0.38763-2.05307-0.37000.h5
Epoch 2/30
23/23 [==============================] - 86s 4s/step - loss: 1.1665 - categorical_accuracy: 0.5572 - val_loss: 1.7547 - val_categorical_accuracy: 0.5100

Epoch 00002: saving model to model_init_2021-02-0807_43_26.250032/model-00002-1.20559-0.53997-1.75467-0.51000.h5
Epoch 3/30
23/23 [==============================] - 88s 4s/step - loss: 1.0020 - categorical_accuracy: 0.6333 - val_loss: 0.9932 - val_categorical_accuracy: 0.6700

Epoch 00003: saving model to model_init_2021-02-0807_43_26.250032/model-00003-0.97345-0.63198-0.99318-0.67000.h5
Epoch 4/30
23/23 [==============================] - 89s 4s/step - loss: 0.8070 - categorical_accuracy: 0.6899 - val_loss: 0.8746 - val_categorical_accuracy: 0.6700

Epoch 00004

### Conclusion : The accuracy increased to 93/87 % with number of epochs but execution time also increases.

### Model 5 - New Architecture with Reduced Parameters

In [15]:
class ModelConv3D5(ModelBuilder):
    
    def define_model(self,dense_neurons=32,dropout=0.25):

        model = Sequential()
        model.add(Conv3D(16, (3, 3, 3), padding='same',
                 input_shape=(self.frames_to_sample,self.image_height,self.image_width,self.channels)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))

        model.add(Conv3D(32, (2, 2, 2), padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))

        model.add(Conv3D(64, (2, 2, 2), padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))

        model.add(Conv3D(128, (2, 2, 2), padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))
        
        model.add(Flatten())
        model.add(Dense(dense_neurons,activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(dropout))

        model.add(Dense(dense_neurons,activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(dropout))

        model.add(Dense(self.num_classes,activation='softmax'))

        optimiser = optimizers.Adam(lr=0.0002)
        model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
        return model

In [25]:
conv_3d5=ModelConv3D5()
conv_3d5.initialize_path(project_folder)
conv_3d5.initialize_image_properties(image_height=160,image_width=160)
conv_3d5.initialize_hyperparams(frames_to_sample=30,batch_size=30,num_epochs=10)
conv_3d5_model=conv_3d5.define_model()
conv_3d5_model.summary()
print("Total Params:", conv_3d5_model.count_params())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_21 (Conv3D)           (None, 30, 160, 160, 16)  1312      
_________________________________________________________________
activation_21 (Activation)   (None, 30, 160, 160, 16)  0         
_________________________________________________________________
batch_normalization_31 (Batc (None, 30, 160, 160, 16)  64        
_________________________________________________________________
max_pooling3d_21 (MaxPooling (None, 15, 80, 80, 16)    0         
_________________________________________________________________
conv3d_22 (Conv3D)           (None, 15, 80, 80, 32)    4128      
_________________________________________________________________
activation_22 (Activation)   (None, 15, 80, 80, 32)    0         
_________________________________________________________________
batch_normalization_32 (Batc (None, 15, 80, 80, 32)    128       
__________

In [26]:
history_model5 = conv_3d5.train_model(conv_3d5_model)

Epoch 1/10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:55: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:56: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


23/23 [==============================] - 100s 4s/step - loss: 1.8192 - categorical_accuracy: 0.3290 - val_loss: 1.3000 - val_categorical_accuracy: 0.4600

Epoch 00001: saving model to model_init_2021-02-0808_45_11.011168/model-00001-1.79243-0.32881-1.29996-0.46000.h5
Epoch 2/10
23/23 [==============================] - 84s 4s/step - loss: 1.3224 - categorical_accuracy: 0.4901 - val_loss: 1.4540 - val_categorical_accuracy: 0.4700

Epoch 00002: saving model to model_init_2021-02-0808_45_11.011168/model-00002-1.28538-0.49623-1.45398-0.47000.h5
Epoch 3/10
23/23 [==============================] - 86s 4s/step - loss: 1.1805 - categorical_accuracy: 0.5476 - val_loss: 1.4244 - val_categorical_accuracy: 0.4400

Epoch 00003: saving model to model_init_2021-02-0808_45_11.011168/model-00003-1.19138-0.54299-1.42437-0.44000.h5
Epoch 4/10
23/23 [==============================] - 86s 4s/step - loss: 0.9798 - categorical_accuracy: 0.6376 - val_loss: 1.2101 - val_categorical_accuracy: 0.5400

Epoch 00004

### Conclusion : Reducing the number of Parameters has increased accuracy significantly .
Accuracy with 10 Epochs with model having more parameters is far less than that with model having less parameters.

### Model 6 : Increase the Epochs to 30 in the model with reduced parameters.

In [29]:
### Model 6 - Same Architecture as previous with  increase in Epochs.

conv_3d6=ModelConv3D5()
conv_3d6.initialize_path(project_folder)
conv_3d6.initialize_image_properties(image_height=160,image_width=160)
conv_3d6.initialize_hyperparams(frames_to_sample=30,batch_size=30,num_epochs=30)
conv_3d6_model=conv_3d6.define_model()
conv_3d6_model.summary()
print("Total Params:", conv_3d6_model.count_params())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_29 (Conv3D)           (None, 30, 160, 160, 16)  1312      
_________________________________________________________________
activation_29 (Activation)   (None, 30, 160, 160, 16)  0         
_________________________________________________________________
batch_normalization_43 (Batc (None, 30, 160, 160, 16)  64        
_________________________________________________________________
max_pooling3d_29 (MaxPooling (None, 15, 80, 80, 16)    0         
_________________________________________________________________
conv3d_30 (Conv3D)           (None, 15, 80, 80, 32)    4128      
_________________________________________________________________
activation_30 (Activation)   (None, 15, 80, 80, 32)    0         
_________________________________________________________________
batch_normalization_44 (Batc (None, 15, 80, 80, 32)    128       
__________

In [30]:
history_model6 = conv_3d6.train_model(conv_3d6_model)

Epoch 1/30


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:55: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:56: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


23/23 [==============================] - 100s 4s/step - loss: 1.8362 - categorical_accuracy: 0.3101 - val_loss: 1.7054 - val_categorical_accuracy: 0.3900

Epoch 00001: saving model to model_init_2021-02-0811_18_15.817203/model-00001-1.84071-0.30920-1.70541-0.39000.h5
Epoch 2/30
23/23 [==============================] - 84s 4s/step - loss: 1.2275 - categorical_accuracy: 0.5157 - val_loss: 1.2813 - val_categorical_accuracy: 0.4400

Epoch 00002: saving model to model_init_2021-02-0811_18_15.817203/model-00002-1.23580-0.50980-1.28127-0.44000.h5
Epoch 3/30
23/23 [==============================] - 88s 4s/step - loss: 1.0214 - categorical_accuracy: 0.5917 - val_loss: 1.1176 - val_categorical_accuracy: 0.6100

Epoch 00003: saving model to model_init_2021-02-0811_18_15.817203/model-00003-0.97327-0.60181-1.11760-0.61000.h5
Epoch 4/30
23/23 [==============================] - 88s 4s/step - loss: 0.9497 - categorical_accuracy: 0.6391 - val_loss: 1.2240 - val_categorical_accuracy: 0.5200

Epoch 00004

### Conclusion : Model seems to overfit. Training Acuracy is in 90s but Validation Accuracy is in 70s.

### Model 7 :with augmentation
Frames : 20, Batch Size : 20, Number of Epochs : 10

In [19]:
conv_3d7=ModelConv3D5()
conv_3d7.initialize_path(project_folder)
conv_3d7.initialize_image_properties(image_height=160,image_width=160)
conv_3d7.initialize_hyperparams(frames_to_sample=20,batch_size=20,num_epochs=10)
conv_3d7_model=conv_3d7.define_model()

conv_3d7_model.summary()



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_5 (Conv3D)            (None, 20, 160, 160, 16)  1312      
_________________________________________________________________
activation_5 (Activation)    (None, 20, 160, 160, 16)  0         
_________________________________________________________________
batch_normalization_7 (Batch (None, 20, 160, 160, 16)  64        
_________________________________________________________________
max_pooling3d_5 (MaxPooling3 (None, 10, 80, 80, 16)    0         
_________________________________________________________________
conv3d_6 (Conv3D)            (None, 10, 80, 80, 32)    4128      
_________________________________________________________________
activation_6 (Activation)    (None, 10, 80, 80, 32)    0         
_________________________________________________________________
batch_normalization_8 (Batch (None, 10, 80, 80, 32)    128       
__________

In [22]:
print("Total Params:", conv_3d7_model.count_params())
history_model7=conv_3d7.train_model(conv_3d7_model,augment_data=True)

Total Params: 499621
Epoch 1/10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:55: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:56: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:75: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


34/34 [==============================] - 235s 7s/step - loss: 1.7735 - categorical_accuracy: 0.3072 - val_loss: 1.1699 - val_categorical_accuracy: 0.5300

Epoch 00001: saving model to model_init_2021-02-0812_52_41.145224/model-00001-1.76265-0.31071-1.16986-0.53000.h5
Epoch 2/10
34/34 [==============================] - 128s 4s/step - loss: 1.2838 - categorical_accuracy: 0.4882 - val_loss: 1.1999 - val_categorical_accuracy: 0.5600

Epoch 00002: saving model to model_init_2021-02-0812_52_41.145224/model-00002-1.28640-0.48793-1.19994-0.56000.h5
Epoch 3/10
34/34 [==============================] - 130s 4s/step - loss: 1.1034 - categorical_accuracy: 0.5784 - val_loss: 1.2040 - val_categorical_accuracy: 0.5200

Epoch 00003: saving model to model_init_2021-02-0812_52_41.145224/model-00003-1.09692-0.57617-1.20404-0.52000.h5
Epoch 4/10
34/34 [==============================] - 129s 4s/step - loss: 1.0612 - categorical_accuracy: 0.5754 - val_loss: 0.9630 - val_categorical_accuracy: 0.6800

Epoch 00

### Accuracy has not improved with Augmentation.